# 2025 Day 3

In [1]:
import os
from getpass import getpass

import aocd
import numpy as np
import pandas as pd
from pydantic import BaseModel

In [2]:
if not os.getenv('AOC_SESSION'):
    os.environ['AOC_SESSION'] = getpass('AOC_SESSION:')

## \--- Day 3: Lobby ---

You descend a short staircase, enter the surprisingly vast lobby, and are quickly cleared by the security checkpoint. When you get to the main elevators, however, you discover that each one has a red light above it: they're all _offline_.

"Sorry about that," an Elf apologizes as she tinkers with a nearby control panel. "Some kind of electrical surge seems to have fried them. I'll try to get them online soon."

You explain your need to get further underground. "Well, you could at least take the escalator down to the printing department, not that you'd get much further than that without the elevators working. That is, you could if the escalator weren't also offline."

"But, don't worry! It's not fried; it just needs power. Maybe you can get it running while I keep working on the elevators."

There are batteries nearby that can supply emergency power to the escalator for just such an occasion. The batteries are each labeled with their [joltage](/2020/day/10) rating, a value from `1` to `9`. You make a note of their joltage ratings (your puzzle input). For example:

```
987654321111111
811111111111119
234234234234278
818181911112111
```

The batteries are arranged into _banks_; each line of digits in your input corresponds to a single bank of batteries. Within each bank, you need to turn on _exactly two_ batteries; the joltage that the bank produces is equal to the number formed by the digits on the batteries you've turned on. For example, if you have a bank like `12345` and you turn on batteries `2` and `4`, the bank would produce `24` jolts. (You cannot rearrange batteries.)

You'll need to find the largest possible joltage each bank can produce. In the above example:

*   In `_98_7654321111111`, you can make the largest joltage possible, _`98`_, by turning on the first two batteries.
*   In `_8_1111111111111_9_`, you can make the largest joltage possible by turning on the batteries labeled `8` and `9`, producing _`89`_ jolts.
*   In `2342342342342_78_`, you can make _`78`_ by turning on the last two batteries (marked `7` and `8`).
*   In `818181_9_1111_2_111`, the largest joltage you can produce is _`92`_.

The total output joltage is the sum of the maximum joltage from each bank, so in this example, the total output joltage is `98` + `89` + `78` + `92` = `_357_`.

There are many batteries in front of you. Find the maximum joltage possible from each bank; _what is the total output joltage?_

In [3]:
puzzle_3 = aocd.models.Puzzle(day=3, year=2025)
puzzle_3.examples[0].input_data

'987654321111111\n811111111111119\n234234234234278\n818181911112111'

In [18]:
def solve_part_1(puzzle_input: str) -> int:
    arr = np.array([[int(d) for d in line] for line in puzzle_input.split()])
    # print("Input")
    # print(arr)
    # Find the max of the first n-1 columns, by row
    # print("First n-1 columns")
    # print(arr[:, :-1])
    max_values = arr[:, :-1].max(axis=1)
    max_indices = arr[:, :-1].argmax(axis=1)
    # print(max_values)
    # print(max_indices)
    col_indices = np.arange(arr.shape[1])
    valid_mask = col_indices > max_indices[:, None]
    max_remaining_values = np.where(valid_mask, arr, -np.inf).max(axis=1)
    # print("Result")
    # print(col_indices)
    # print(valid_mask)
    # print(max_remaining_values)
    result = (10 * max_values + max_remaining_values).sum().astype(int)
    return result.item()

    

solve_part_1(puzzle_3.examples[0].input_data)

357

In [19]:
solve_part_1(puzzle_3.input_data)

17142

## \--- Part Two ---

The escalator doesn't move. The Elf explains that it probably needs more joltage to overcome the [static friction](https://en.wikipedia.org/wiki/Static_friction) of the system and hits the big red "joltage limit safety override" button. You lose count of the number of times she needs to confirm "yes, I'm sure" and decorate the lobby a bit while you wait.

Now, you need to make the largest joltage by turning on _exactly twelve_ batteries within each bank.

The joltage output for the bank is still the number formed by the digits of the batteries you've turned on; the only difference is that now there will be `_12_` digits in each bank's joltage output instead of two.

Consider again the example from before:

```
987654321111111
811111111111119
234234234234278
818181911112111
```

Now, the joltages are much larger:

*   In `_987654321111_111`, the largest joltage can be found by turning on everything except some `1`s at the end to produce `_987654321111_`.
*   In the digit sequence `_81111111111_111_9_`, the largest joltage can be found by turning on everything except some `1`s, producing `_811111111119_`.
*   In `23_4_2_34234234278_`, the largest joltage can be found by turning on everything except a `2` battery, a `3` battery, and another `2` battery near the start to produce `_434234234278_`.
*   In `_8_1_8_1_8_1_911112111_`, the joltage `_888911112111_` is produced by turning on everything except some `1`s near the front.

The total output joltage is now much larger: `987654321111` + `811111111119` + `434234234278` + `888911112111` = `_3121910778619_`.

_What is the new total output joltage?_

In [66]:
def solve_part_2(puzzle_input: str) -> int:
    n_digits = 12
    arr = np.array([[int(d) for d in line] for line in puzzle_input.splitlines()])
    print("Input array:")
    print(arr)
    n_rows, n_columns = arr.shape
    # print(f"n_rows: {n_rows}, n_columns: {n_columns}")
    assert n_digits <= n_columns
    result_arr = np.zeros((n_rows, n_digits), dtype=int)
    result_idx = np.zeros((n_rows, n_digits), dtype=int)
    col_indices = np.arange(arr.shape[1])
    # print(f"result_arr shape: {result_arr.shape}")
    # print(f"result_idx shape: {result_idx.shape}")
    # print(f"col_indices: {col_indices}")
    # Initialize the max_indices to the first column
    max_indices = np.ones((n_rows,), dtype=int) * -1
    # print(f"max_indices: {max_indices}")
    for column_start in range(n_digits):
        column_end: int = n_columns - n_digits + column_start
        print(f"column_start: {column_start}, column_end: {column_end}")
        # Initialize the valid_mask:
        # - False if it is less than or equal to max_indices
        # - False if it is greater than (n_columns - n_digits + column_start)
        # - True otherwise
        valid_mask = np.ones((n_rows, n_columns), dtype=bool)
        valid_mask = np.where(col_indices <= max_indices[:, None], False, valid_mask)
        valid_mask = np.where(col_indices > (n_columns - n_digits + column_start), False, valid_mask)
        # print(f"valid_mask:\n{valid_mask.astype(int)}")
        # Calculate the max values based on the current mask
        max_values = np.where(valid_mask, arr, -np.inf).max(axis=1)
        # print(f"max_values: {max_values}, shape: {max_values.shape}")
        max_indices = np.where(valid_mask, arr, -np.inf).argmax(axis=1)
        # print(f"max_indices: {max_indices}, shape: {max_indices.shape}")
        result_arr[:, column_start] = max_values
        result_idx[:, column_start] = max_indices
    # Multiply each element by 10**(n_digits - column_start - 1)
    result = np.sum(result_arr * np.power(10, np.arange(n_digits - 1, -1, -1)), axis=1).sum().item()
    return result
    


solve_part_2(puzzle_3.examples[0].input_data)

Input array:
[[9 8 7 6 5 4 3 2 1 1 1 1 1 1 1]
 [8 1 1 1 1 1 1 1 1 1 1 1 1 1 9]
 [2 3 4 2 3 4 2 3 4 2 3 4 2 7 8]
 [8 1 8 1 8 1 9 1 1 1 1 2 1 1 1]]
column_start: 0, column_end: 3
column_start: 1, column_end: 4
column_start: 2, column_end: 5
column_start: 3, column_end: 6
column_start: 4, column_end: 7
column_start: 5, column_end: 8
column_start: 6, column_end: 9
column_start: 7, column_end: 10
column_start: 8, column_end: 11
column_start: 9, column_end: 12
column_start: 10, column_end: 13
column_start: 11, column_end: 14


3121910778619

In [68]:
solve_part_2(puzzle_3.input_data)

Input array:
[[2 2 5 ... 1 3 2]
 [4 2 4 ... 7 8 9]
 [2 3 4 ... 5 5 5]
 ...
 [5 2 4 ... 2 3 2]
 [8 8 9 ... 7 4 6]
 [6 3 2 ... 6 2 5]]
column_start: 0, column_end: 88
column_start: 1, column_end: 89
column_start: 2, column_end: 90
column_start: 3, column_end: 91
column_start: 4, column_end: 92
column_start: 5, column_end: 93
column_start: 6, column_end: 94
column_start: 7, column_end: 95
column_start: 8, column_end: 96
column_start: 9, column_end: 97
column_start: 10, column_end: 98
column_start: 11, column_end: 99


169935154100102